Ստորև գրված առաջադրանքը կատարելու վերջնաժամկետը **13.05.2023 23:59** է։ Առաջադրանքը կատարելուց հետո, պետք է 

1. ներբեռնեք այս ֆայլը ձեր համակարգիչ (`File` $\to$ `Download .ipynb`)
2. վերնագրին ավելացնեք ձեր անուն, ազգանունը լատինատառ, օրինակ՝ *Bonus_HW_NshanPotikyan.ipynb* ֆորմատով
3. ուղարկեք ֆայլը **nshan.potikyan@gmail.com** հասցեին` նամակի թեմա (subject) դաշտում գրելով **Bonus**

**Ուշադրություն**․ 
1. Վերոնշյալ պայմաններին չբավարարելու դեպքում ձեր աշխատանքը չի գնահատվի։
2. **numpy**, **pandas**, **sklearn** գրադարաններից բացի այլ գրադարաններից չի՛ կարելի օգտվել 
3. Կոդի համար նախատեսված հավելյալ վանդակներ ստեղծել, ինչպես նաև եղած վանդակների հերթականությունը փոխել չի՛ կարելի  
4. Կրկնօրինակ լուծումների համար կստանաք 0 միավոր։


**Հավելյալ (Ճիշտ լուծելու դեպքում կստանաք 1/20 միավոր)** 

Դիցուք՝ ցանկանում ենք կառուցել այնպիսի դասակարգիչ (classifier), որը հնարավորություն կտա յուրաքանչյուր ուսանողի համար գնահատել նրա կողմից ինչ֊որ հարցի ճիշտ պատասխանելու հավանականությունը՝ հաշվի առնելով այդ ուսանողի տվյալները և հարցի բարդությունը։ 

Ուսանողների տվյալներից օգտագործվելու է նրա 
* սեռը (gender),
* նախորդ հարցերին պատասխանելու միջին ժամանակը (avg_time), 
* նախորդ հարցերին ճիշտ պատասխանելու միջին համամասնությունը (avg_correct) 

Որպես հարցի բարդության նկարագիր, կօգտագործենք

* հարցին հատկացված ժամանակը (total_time)
* հարցի բովանդակության մեջ եղած բառերի քանակը (question)
* հարցի պատասխանի 4 տարբերակների մեջ առկա բառերի քանակների գումարը (answer1,2,3,4)

Որպես պիտակ (label) կօգտագործենք correct սյան տվյալները:

Պահանջվում է լրացնել function6-ը այնպես, որ այն իր մեջ կառուցի sklearn.pipeline.Pipeline, հետևյալ փուլերով․
* gender սյան վրա կիրառվող sklearn.preprocessing.OrdinalEncoder
* question սյան վրա կիրառվող sklearn.preprocessing.FunctionTransformer` այնպիսի ֆունկցիայով, որով կվերադարձվի բառերի քանակը անտեսելով տառերից բացի այլ նիշերը [[հուշում]](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)
* answer1,2,3,4 սյուների վրա կիրառվող sklearn.preprocessing.FunctionTransformer` այնպիսի ֆունկցիայով, որով կվերադարձվի 4 տարբերակների մեջ եղած բառերի քանակի (հաշվել նախորդ կետի նման) գումարը
* այնուհետև մուտքային տվյալների (X) բոլոր սյուների վրա կիրառել sklearn.preprocessing.StandardScaler
* և որպես դասակարգիչ  օգտագործել sklearn.neighbors․KNeighborsClassifier-ը k=7 արժեքով

Առանձին սյուների վրա ձևափոխություն անելու համար կարող եք օգտագործել [սա](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn-compose-columntransformer)։ Պահանջվում է վերադարձնել train (``hw4_bonus_train.csv``) բազմության վրա ուսուցանված pipeline-ի օբյեկտը և test (``hw4_bonus_test.csv``) բազմության վրա կանխատեսումների ճշգրտությունը (կարելի է օգտագործել score մեթոդը)։



**Ներբեռնել տվյալները** 🔽

In [2]:
!wget https://raw.githubusercontent.com/NshanPotikyan/Dasa1Doom/master/files/hw4_bonus.zip
!unzip hw4_bonus.zip

--2023-05-13 11:22:17--  https://raw.githubusercontent.com/NshanPotikyan/Dasa1Doom/master/files/hw4_bonus.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34872 (34K) [application/zip]
Saving to: ‘hw4_bonus.zip’

hw4_bonus.zip       100%[===================>]  34.05K  --.-KB/s    in 0.002s  

2023-05-13 11:22:17 (17.2 MB/s) - ‘hw4_bonus.zip’ saved [34872/34872]

Archive:  hw4_bonus.zip
  inflating: hw4_bonus_train.csv     
  inflating: hw4_bonus_test.csv      


In [94]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer

def count_words(D):
  X = D.apply(' '.join,axis=1)
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(X)
  return np.asarray(X.sum(axis=1).reshape(-1, 1))


def function(train,test):

  features = ['gender', 'avg_time', 'avg_correct', 'total_time', 'question', 'answer1', 'answer2', 'answer3', 'answer4']
  gender_encoder = ('gender_encoder', OrdinalEncoder(), ['gender'])
  question_word_count = ('question_word_count', FunctionTransformer(lambda X: count_words(X[['question']])), ['question'])
  answer_word_count = ('answer_word_count', FunctionTransformer(lambda X: count_words(X[['answer1', 'answer2', 'answer3', 'answer4']])),['answer1', 'answer2', 'answer3', 'answer4'])

  preprocessor = ColumnTransformer(transformers=[
      gender_encoder,
      question_word_count,
      answer_word_count
  ], remainder='passthrough')
  
  pipeline = Pipeline([
      ('preprocessor', preprocessor),
      ('scaler', StandardScaler()),
      ('clf', KNeighborsClassifier(n_neighbors=7))
  ])

  X_train = train[features]
  y_train = train['correct']
  pipeline.fit(X_train, y_train)
  
  X_test = test[features]
  y_test = test['correct']
  accuracy_score = pipeline.score(X_test, y_test)

  return pipeline, accuracy_score

In [ ]:
# այստեղ կարող եք ստուգել ձեր լուծումը
train = pd.read_csv('hw4_bonus_train.csv')
test = pd.read_csv('hw4_bonus_test.csv')
pipeline,accuracy = function(train,test)
print('Prediction Accuracy:', accuracy)
print('Pipeline: ',pipeline)